Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

# Inference PyTorch Bert Model with ONNX Runtime on CPU

In this tutorial, you'll be introduced to how to load a Bert model from PyTorch, convert it to ONNX, and inference it for high performance using ONNX Runtime. In the following sections, we are going to use the Bert model trained with Stanford Question Answering Dataset (SQuAD) dataset as an example. Bert SQuAD model is used in question answering scenarios, where the answer to every question is a segment of text, or span, from the corresponding reading passage, or the question might be unanswerable.

This notebook is for CPU inference. For GPU inferenece, please look at another notebook [Inference PyTorch Bert Model with ONNX Runtime on GPU](PyTorch_Bert-Squad_OnnxRuntime_GPU.ipynb).

## 0. Prerequisites ##

If you have Jupyter Notebook, you may directly run this notebook. We will use pip to install or upgrade [PyTorch](https://pytorch.org/), [OnnxRuntime](https://microsoft.github.io/onnxruntime/) and other required packages.

Otherwise, you can setup a new environment. First, we install [AnaConda](https://www.anaconda.com/distribution/). Then open an AnaConda prompt window and run the following commands:

```console
conda create -n cpu_env python=3.10
conda activate cpu_env
pip install jupyterlab
conda install ipykernel
conda install -c conda-forge ipywidgets
ipython kernel install --user --name cpu_env
jupyter-lababook
```
The last command will launch Jupyter Notebook and we can open this notebook in browser to continue.

In [1]:
"""
# Please refer to https://pytorch.org to install CPU-only PyTorch
import sys
if sys.platform in ["darwin", "win32"]:  # Mac or Windows
    !{sys.executable} -m pip install torch -q
else:
    !{sys.executable} -m pip install install torch --index-url https://download.pytorch.org/whl/cpu -q

# Install other packages used in this notebook.
!{sys.executable} -m pip install onnxruntime onnx transformers==4.18 psutil pandas py-cpuinfo py3nvml wget netron -q
"""

'\n# Please refer to https://pytorch.org to install CPU-only PyTorch\nimport sys\nif sys.platform in ["darwin", "win32"]:  # Mac or Windows\n    !{sys.executable} -m pip install torch -q\nelse:\n    !{sys.executable} -m pip install install torch --index-url https://download.pytorch.org/whl/cpu -q\n\n# Install other packages used in this notebook.\n!{sys.executable} -m pip install onnxruntime onnx transformers==4.18 psutil pandas py-cpuinfo py3nvml wget netron -q\n'

## 1. Load Pretrained Bert model ##

We begin by downloading the SQuAD data file and store them in the specified location.

In [5]:
import os

cache_dir = os.path.join(".", "cache_models")
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

predict_file_url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json"
predict_file = os.path.join(cache_dir, "dev-v1.1.json")
"""
if not os.path.exists(predict_file):
    import wget
    print("Start downloading predict file.")
    wget.download(predict_file_url, predict_file)
    print("Predict file downloaded.")
"""

'\nif not os.path.exists(predict_file):\n    import wget\n    print("Start downloading predict file.")\n    wget.download(predict_file_url, predict_file)\n    print("Predict file downloaded.")\n'

Specify some model configuration variables and constant.

In [6]:
# For fine tuned large model, the model name is "bert-large-uncased-whole-word-masking-finetuned-squad". Here we use bert-base for demo.
model_name_or_path = "bert-base-cased"
max_seq_length = 128
doc_stride = 128
max_query_length = 64

# Enable overwrite to export onnx model and download latest script each time when running this notebook.
enable_overwrite = True

# Total samples to inference. It shall be large enough to get stable latency measurement.
total_samples = 100

Start to load model from pretrained. This step could take a few minutes. 

In [24]:
# The following code is adapted from HuggingFace transformers
# https://github.com/huggingface/transformers/blob/master/examples/run_squad.py

%env HF_ENDPOINT=https://hf-mirror.com
%env HF_HUB_ENABLE_HF_TRANSFER=1

from transformers import (BertConfig, BertForQuestionAnswering, BertTokenizer)

# Load pretrained model and tokenizer
config_class, model_class, tokenizer_class = (BertConfig, BertForQuestionAnswering, BertTokenizer)
config = config_class.from_pretrained(model_name_or_path, cache_dir=cache_dir)
tokenizer = tokenizer_class.from_pretrained(model_name_or_path, do_lower_case=True, cache_dir=cache_dir)
model = model_class.from_pretrained(model_name_or_path,
                                    from_tf=False,
                                    config=config,
                                    cache_dir=cache_dir)
# load some examples
from transformers.data.processors.squad import SquadV1Processor

processor = SquadV1Processor()
examples = processor.get_dev_examples(None, filename=predict_file)

from transformers import squad_convert_examples_to_features
features, dataset = squad_convert_examples_to_features(
            examples=examples[:total_samples], # convert just enough examples for this notebook
            tokenizer=tokenizer,
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_training=False,
            return_dataset='pt'
        )

env: HF_ENDPOINT=https://hf-mirror.com
env: HF_HUB_ENABLE_HF_TRANSFER=1


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
add example index and unique id: 100%|██████████| 100/100 [00:00<00:00, 252668.92it/s]


## 2. Export the loaded model ##
Once the model is loaded, we can export the loaded PyTorch model to ONNX.

In [9]:
output_dir = os.path.join(".", "onnx_models")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
export_model_path = os.path.join(output_dir, 'bert-base-cased-squad.onnx')

import torch
device = torch.device("cpu")

# Get the first example data to run the model and export it to ONNX
data = dataset[0]
inputs = {
    'input_ids':      data[0].to(device).reshape(1, max_seq_length),
    'attention_mask': data[1].to(device).reshape(1, max_seq_length),
    'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
}

# Set model to inference mode, which is required before exporting the model because some operators behave differently in
# inference and training mode.
model.eval()
model.to(device)

if enable_overwrite or not os.path.exists(export_model_path):
    with torch.no_grad():
        symbolic_names = {0: 'batch_size', 1: 'max_seq_len'}
        torch.onnx.export(model,                                            # model being run
                          args=tuple(inputs.values()),                      # model input (or a tuple for multiple inputs)
                          f=export_model_path,                              # where to save the model (can be a file or file-like object)
                          opset_version=14,                                 # the ONNX version to export the model to
                          do_constant_folding=True,                         # whether to execute constant folding for optimization
                          input_names=['input_ids',                         # the model's input names
                                       'input_mask',
                                       'segment_ids'],
                          output_names=['start', 'end'],                    # the model's output names
                          dynamic_axes={'input_ids': symbolic_names,        # variable length axes
                                        'input_mask' : symbolic_names,
                                        'segment_ids' : symbolic_names,
                                        'start' : symbolic_names,
                                        'end' : symbolic_names})
        print("Model exported at ", export_model_path)

Model exported at  ./onnx_models/bert-base-cased-squad.onnx


## 3. PyTorch Inference ##
Use PyTorch to evaluate an example input for comparison purpose.

In [10]:
import time

# Measure the latency. It is not accurate using Jupyter Notebook, it is recommended to use standalone python script.
latency = []
with torch.no_grad():
    for i in range(total_samples):
        data = dataset[i]
        inputs = {
            'input_ids':      data[0].to(device).reshape(1, max_seq_length),
            'attention_mask': data[1].to(device).reshape(1, max_seq_length),
            'token_type_ids': data[2].to(device).reshape(1, max_seq_length)
        }
        start = time.time()
        outputs = model(**inputs)
        latency.append(time.time() - start)
print("PyTorch {} Inference time = {} ms".format(device.type, format(sum(latency) * 1000 / len(latency), '.2f')))

PyTorch cpu Inference time = 57.95 ms


## 4. Inference ONNX Model with ONNX Runtime ##

For Onnx Runtime 1.6.0 or older, OpenMP environment variables are very important for CPU inference of Bert model. Since 1.7.0, the official package is not built with OpenMP.

Now we inference the model with ONNX Runtime. Here we can see that OnnxRuntime has better performance than PyTorch. 

In [11]:
import onnxruntime
import numpy

sess_options = onnxruntime.SessionOptions()

# Optional: store the optimized graph and view it using Netron to verify that model is fully optimized.
# Note that this will increase session creation time, so it is for debugging only.
sess_options.optimized_model_filepath = os.path.join(output_dir, "optimized_model_cpu.onnx")

# Specify providers when you use onnxruntime-gpu for CPU inference.
session = onnxruntime.InferenceSession(export_model_path, sess_options, providers=['CPUExecutionProvider'])

latency = []
for i in range(total_samples):
    data = dataset[i]
    ort_inputs = {
        'input_ids':  data[0].cpu().reshape(1, max_seq_length).numpy(),
        'input_mask': data[1].cpu().reshape(1, max_seq_length).numpy(),
        'segment_ids': data[2].cpu().reshape(1, max_seq_length).numpy()
    }
    start = time.time()
    ort_outputs = session.run(None, ort_inputs)
    latency.append(time.time() - start)
print("OnnxRuntime cpu Inference time = {} ms".format(format(sum(latency) * 1000 / len(latency), '.2f')))

2025-01-11 18:02:32.939794600 [W:onnxruntime:, inference_session.cc:2048 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.


OnnxRuntime cpu Inference time = 40.93 ms


In [12]:
print("***** Verifying correctness *****")
for i in range(2):
    print('PyTorch and ONNX Runtime output {} are close:'.format(i), numpy.allclose(ort_outputs[i], outputs[i].cpu(), rtol=1e-05, atol=1e-04))

***** Verifying correctness *****
PyTorch and ONNX Runtime output 0 are close: True
PyTorch and ONNX Runtime output 1 are close: True


## 5. Offline Optimization Script and Test Tools

It is recommended to try [OnnxRuntime Transformer Model Optimization Tool](https://github.com/microsoft/onnxruntime/blob/main/onnxruntime/python/tools/transformers) on the exported ONNX models. It could help verify whether the model can be fully optimized, and get performance test results.

#### Transformer Optimizer

Although OnnxRuntime could optimize Bert model exported by PyTorch. Sometime, model cannot be fully optimized due to different reasons:
* A new subgraph pattern is generated by new version of export tool, and the pattern is not covered by older version of OnnxRuntime. 
* The exported model uses dynamic axis and this makes it harder for shape inference of the graph. That blocks some optimization to be applied.
* Some optimization is better to be done offline. Like change input tensor type from int64 to int32 to avoid extra Cast nodes, or convert model to float16 to achieve better performance in V100 or T4 GPU.

We have python script **optimizer.py**, which is more flexible in graph pattern matching and model conversion (like float32 to float16). You can also use it to verify whether a Bert model is fully optimized.

In this example, we can see that it introduces optimization that is not provided by onnxruntime: SkipLayerNormalization and bias fusion, which is not fused in OnnxRuntime due to shape inference as mentioned.

It will also tell whether the model is fully optimized or not. If not, that means you might need change the script to fuse some new pattern of subgraph.

Example Usage:
```
from onnxruntime.transformers import optimizer
optimized_model = optimizer.optimize_model(export_model_path, model_type='bert', num_heads=12, hidden_size=768)
optimized_model.save_model_to_file(optimized_model_path)
```

You can also use command line like the following:

In [16]:
import sys

optimized_model_path = os.path.join(output_dir, 'bert-base-cased-squad_opt_cpu.onnx')

!{sys.executable} -m onnxruntime.transformers.optimizer --input $export_model_path --output $optimized_model_path --model_type bert

               apply: Fused LayerNormalization: 25
               apply: Fused Gelu: 12
               apply: Fused SkipLayerNormalization: 24
remove_useless_reshape_nodes: Remove reshape node /bert/Reshape since its input shape is same as output: [4]
               apply: Fused BiasGelu: 12
               apply: Fused SkipLayerNormalization(add bias): 24
            optimize: opset version: 14
get_operator_statistics: Operators:[('MatMul', 97), ('Add', 51), ('Reshape', 48), ('Transpose', 48), ('Gather', 42), ('Unsqueeze', 30), ('Shape', 26), ('Mul', 24), ('SkipLayerNormalization', 24), ('Sqrt', 24), ('Cast', 14), ('Concat', 13), ('Slice', 13), ('BiasGelu', 12), ('Div', 12), ('Softmax', 12), ('Squeeze', 2), ('Where', 2), ('Equal', 1), ('Expand', 1), ('LayerNormalization', 1), ('Split', 1), ('Sub', 1)]
get_fused_operator_statistics: Optimized operators: {'EmbedLayerNormalization': 0, 'Attention': 0, 'MultiHeadAttention': 0, 'Gelu': 0, 'FastGelu': 0, 'BiasGelu': 12, 'GemmFastGelu': 0, 'L

#### Optimized Graph
When you can open the optimized model using Netron to visualize, the graph is like the following:
<img src='images/optimized_bert_gpu.png'>

For CPU, optimized graph is slightly different: FastGelu is replaced by BiasGelu.

In [17]:
import netron

# Change it to False to skip viewing the optimized model in browser.
enable_netron = True
if enable_netron:
    # If you encounter error "access a socket in a way forbidden by its access permissions", install Netron as standalone application instead.
    netron.start(optimized_model_path)

Serving './onnx_models/bert-base-cased-squad_opt_cpu.onnx' at http://localhost:8080


#### Model Results Comparison Tool

If your BERT model has three inputs, a script compare_bert_results.py can be used to do a quick verification. The tool will generate some fake input data, and compare results from both the original and optimized models. If outputs are all close, it is safe to use the optimized model.

Example of verifying models:

In [19]:
!{sys.executable} -m onnxruntime.transformers.compare_bert_results --baseline_model $export_model_path --optimized_model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100

100% passed for 100 random inputs given thresholds (rtol=0.001, atol=0.0001).
maximum absolute difference=7.741153240203857e-06


#### Performance Test Tool

This tool measures performance of BERT model inference using OnnxRuntime Python API.

The following command will create 100 samples of batch_size 1 and sequence length 128 to run inference, then calculate performance numbers like average latency and throughput etc. You can increase number of samples (recommended 1000) to get more stable result.

In [20]:
!{sys.executable} -m onnxruntime.transformers.bert_perf_test --model $optimized_model_path --batch_size 1 --sequence_length 128 --samples 100 --test_times 1

test setting TestSetting(batch_size=1, sequence_length=128, test_cases=100, test_times=1, use_gpu=False, use_io_binding=False, provider=None, intra_op_num_threads=None, seed=3, verbose=False, log_severity=2, average_sequence_length=128, random_sequence_length=False)
Generating 100 samples for batch_size=1 sequence_length=128
Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=28,batch_size=1,sequence_length=128,test_cases=100,test_times=1,use_gpu=False,use_io_binding=False,average_sequence_length=128,random_sequence_length=False
Average latency = 58.21 ms, Throughput = 17.18 QPS
Running test: model=bert-base-cased-squad_opt_cpu.onnx,graph_optimization_level=ENABLE_ALL,intra_op_num_threads=15,batch_size=1,sequence_length=128,test_cases=100,test_times=1,use_gpu=False,use_io_binding=False,average_sequence_length=128,random_sequence_length=False
Average latency = 53.28 ms, Throughput = 18.77 QPS
Running test: model=bert-base-cased

Let's load the summary file and take a look.

In [21]:
import os
import glob
import pandas
latest_result_file = max(glob.glob(os.path.join(output_dir, "perf_results_*.txt")), key=os.path.getmtime)
result_data = pandas.read_table(latest_result_file, converters={'OMP_NUM_THREADS': str, 'OMP_WAIT_POLICY':str})
print(latest_result_file)

# Remove some columns that have same values for all rows.
columns_to_remove = ['model', 'graph_optimization_level', 'batch_size', 'sequence_length', 'test_cases', 'test_times', 'use_gpu']
# Hide some latency percentile columns to fit screen width.
columns_to_remove.extend(['Latency_P50', 'Latency_P95'])
result_data.drop(columns_to_remove, axis=1, inplace=True)
result_data

./onnx_models/perf_results_CPU_B1_S128_20250111-181820.txt


,Latency(ms),Latency_P75,Latency_P90,Latency_P99,Throughput(QPS),intra_op_num_threads,use_io_binding,average_sequence_length,random_sequence_length
0,37.45,37.39,37.79,45.16,26.70,14,False,128,False
1,38.08,37.27,38.13,55.51,26.26,13,False,128,False
2,39.03,39.07,39.63,46.51,25.62,12,False,128,False
3,43.31,43.49,43.98,50.27,23.09,11,False,128,False
4,44.37,44.60,45.57,48.79,22.54,10,False,128,False
5,47.71,47.67,48.26,53.16,20.96,9,False,128,False
6,51.83,52.45,52.93,55.18,19.29,8,False,128,False
7,53.28,53.39,53.64,54.25,18.77,15,False,128,False
8,55.55,55.56,55.77,57.75,18.00,7,False,128,False
9,58.21,57.95,59.99,96.54,17.18,28,False,128,False


## 6. Quantization

Please see example in [BERT quantization notebook](https://github.com/microsoft/onnxruntime-inference-examples/blob/main/quantization/notebooks/bert/Bert-GLUE_OnnxRuntime_quantization.ipynb).

## 7. Additional Info

Note that running Jupyter Notebook has slight impact on performance result since Jupyter Notebook is using system resources like CPU and memory etc. It is recommended to close Jupyter Notebook and other applications, then run the performance test tool in a console to get more accurate performance numbers.

We have a [benchmark script](https://github.com/microsoft/onnxruntime/blob/main/onnxruntime/python/tools/transformers/run_benchmark.sh). It is recommended to use it compare inference speed of OnnxRuntime with PyTorch.

[OnnxRuntime C API](https://github.com/microsoft/onnxruntime/blob/main/docs/C_API.md) could get slightly better performance than python API. If you use C API in inference, you can use OnnxRuntime_Perf_Test.exe built from source to measure performance instead.

Here is the machine configuration that generated the above results. The machine has GPU but not used in CPU inference.
You might get slower or faster result based on your hardware.

In [23]:
!{sys.executable} -m onnxruntime.transformers.machine_info --silent

/home/haoyu/anaconda3/envs/vLLM/lib/python3.12/site-packages/onnxruntime/transformers/machine_info.py:126: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
{
  "gpu": {
    "driver_version": "535.183.01",
    "devices": [
      {
        "memory_total": 25769803776,
        "memory_available": 24827854848,
        "name": "NVIDIA GeForce RTX 3090"
      }
    ]
  },
  "cpu": {
    "brand": "Intel(R) Xeon(R) CPU E5-2680 v4 @ 2.40GHz",
    "cores": 14,
    "logical_cores": 28,
    "hz": "1596217000,0",
    "l2_cache": "3.5 MiB",
    "flags": "3dnowprefetch,abm,acpi,adx,aes,aperfmperf,apic,arat,arch_perfmon,avx,avx2,bmi1,bmi2,bts,cat_l3,cdp_l3,clflush,cmov,constant_tsc,cpuid,cpuid_fault,cqm,cqm_llc,cqm_mbm_local,cqm_mbm_total,cqm_occup_llc,cx16,cx8,dca,de,ds_cpl,dtes64,dtherm,dts,epb,ept,ept_ad,erms,est,f16c,flexpriority,flush_l1d,fma,fpu,fsgsbase,fxsr,hle,ht,ibpb,ibrs,ida,intel_ppin,intel_pt,invp